[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rycroft-group/math714/blob/main/h_hyperbolic/hyperbolic.ipynb)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import exp

# Optional: a library for plotting with LaTeX-like 
# styles nicer formatted figures
# Warning: need to have LaTeX installed
import scienceplots
plt.style.use(['science'])

# Hyperbolic equations

## Linear advection equation

We will solve for the linear advection equation
$$
u_t + c u_{x} = 0
$$
with the upwinding method and the central difference method.

### Setup

In [ ]:
# Grid size
m = 64
a = np.empty((m))
b = np.empty((m))
snaps = 40 #100
iters = 10 #40
z = np.empty((m, snaps+1))

# PDE-related constants; try switching c to -0.1 to see the
# unstable scheme
c = 0.1 # try: -0.1 for upwinding
dx = 1.0/m
dt = 0.01
nu = c*dt/dx

# Initial condition
for i in range(m):
    x = dx*i
    a[i] = exp(-20*(x-0.5)**2)
z[:, 0] = a

### Integrate the PDE

In [ ]:
# Integrate the PDE using upwinding method
for i in range(1,snaps+1):
    for k in range(iters):
        for j in range(m):
            jl=j-1
            if jl<0: jl+=m
            b[j]=(1-nu)*a[j]+nu*a[jl]
        a=np.copy(b)
    z[:,i]=a

In [ ]:
# Upwinding
results_up = []
# Output results
for j in range(m):
    e=[str(j*dx)]
    for i in range(snaps+1):
        e.append(str(z[j,i]))
    results_up.append(e)
    print(" ".join(e))

In [ ]:
# Integrate the PDE using centered differencing
for i in range(1,snaps+1):
    for k in range(iters):
        for j in range(m):
            jl=j-1
            if jl<0: jl+=m
            jr=j+1
            if jr>=m: jr-=m

            b[j]=a[j]-0.5*nu*(a[jr]-a[jl])
        a=np.copy(b)
    z[:,i]=a

In [ ]:
# Central difference
results_cd = []
# Output results
for j in range(m):
    e=[str(j*dx)]
    for i in range(snaps+1):
        e.append(str(z[j,i]))
    results_cd.append(e)
    print(" ".join(e))

### Visualize

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6), dpi=300)

results_up = np.array(results_up, dtype=float)
colors = plt.cm.viridis(np.linspace(0, 1, snaps+1))

for i in range(1, snaps+1):
    ax.plot(results_up[:, 0], results_up[:, i],
            label=f't={i*dt*iters:.2f}', color=colors[i])

# Formatting
ax.set_xlabel('$x$')
ax.set_ylabel('$u(x,t)$')
sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=0, vmax=snaps*dt*iters))
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, label='Time')

plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 8), dpi=300)
ax = fig.add_subplot(111, projection='3d')

# Prepare data for the 3D plot
X = np.linspace(0, 1, m)  # spatial dimension
T = np.linspace(0, snaps * dt * iters, snaps + 1)  # time dimension
X, T = np.meshgrid(X, T)
Z = z.T  # transpose z to match the dimensions of X and T

surf = ax.plot_surface(X, T, Z, cmap='inferno', edgecolor='none')

# Formatting
ax.set_xlabel('$x$')
ax.set_ylabel('$t$')
ax.set_zlabel('$u(x,t)$')
fig.colorbar(surf, ax=ax, label='Temperature', shrink=0.5)

plt.show()